<a href="https://colab.research.google.com/github/RaghavPeri/ai-mvp-portfolio/blob/main/legal-rag-chatbot/Legal_RAG_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📄 Legal RAG Chatbot (LangChain + Google Sheets MVP)

This notebook demonstrates a minimal yet scalable **Retrieval-Augmented Generation (RAG)** pipeline using **LangChain**, **OpenAI**, and **Google Sheets**.  
It allows legal questions to be entered into a shared Google Sheet, where answers are automatically generated by retrieving relevant chunks from a legal PDF and sending them to a GPT model for contextual response.

### ✅ What It Covers:
- PDF ingestion and chunking using LangChain
- Vector embedding using OpenAI
- Semantic retrieval via FAISS
- LLM-powered response generation
- Google Sheets integration for input/output
- Conditional row-level automation using `RunnableSequence`

This MVP is optimized for extension into a real-time app or API by swapping out the sheet logic with a form or frontend trigger.


## 🔧  Setup and LangChain RAG Pipeline Initialization

This block:
1. Installs all required dependencies (`langchain`, `langchain-community`, `langchain-openai`, etc.)
2. Loads the uploaded legal document (PDF)
3. Splits the document into overlapping chunks
4. Creates vector embeddings using OpenAI
5. Stores vectors in a FAISS index for fast retrieval
6. Assembles the LangChain `RetrievalQA` pipeline using `OpenAI` LLM

📌 This cell is run once per session to prepare the RAG engine (`qa_chain`) before processing any user questions.


In [4]:
## Step 1: Install LangChain + LangChain-OpenAI (official LLM & embedding classes)
!pip install -q -U langchain langchain-community langchain-openai faiss-cpu openai tiktoken pypdf

## Step 2: Import Required Libraries
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import OpenAIEmbeddings, OpenAI
import os

## Step 3:  Upload a Legal PDF
## Use Colab’s left panel → Files → Click Upload.
## Upload a public legal document, e.g., “Tenant Rights.pdf”

## Step 4: Load the legal PDF
loader = PyPDFLoader("Tenant Rights.pdf")
docs = loader.load()

# Split into manageable chunks
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
chunks = splitter.split_documents(docs)
print(f"Total chunks created: {len(chunks)}")

## Step 5:Embed & Create FAISS Vector Store
# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"  # 🔒 Replace before running


# Create vector embeddings
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(chunks, embeddings)

## Step 6: Build RAG-powered QA Chain
retriever = vectorstore.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(temperature=0),
    chain_type="stuff",
    retriever=retriever
)
print("✅ Step 6 Complete: RAG-powered QA chain successfully built and ready to answer questions.")



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 723.4/723.4 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 27.7 MB/s eta 0:00:00
Total chunks created: 55
✅ Step 6 Complete: RAG-powered QA chain successfully built and ready to answer questions.


## 🧠 RunnableSequence + Google Sheets Integration

This block:
1. Authenticates Google Sheets access via `gspread`
2. Reads questions, answers, and timestamps from the sheet
3. Finds the latest unanswered question
4. Uses a LangChain `RunnableSequence` to:
   - Check if a question needs to be answered
   - Run the `qa_chain` if needed
   - Write the answer and a timestamp (in local time) to the sheet
5. Handles errors gracefully by logging them into a dedicated column

📌 This design ensures the system only processes new questions, leaving prior responses untouched.


In [12]:
## Step 1: Install & Import
!pip install -q gspread oauth2client
import gspread
from google.auth import default
from datetime import datetime
import pytz
from langchain_core.runnables import RunnableLambda, RunnableSequence

## Step 2: Authenticate and connect to Google Sheets
from google.colab import auth
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

## Step 3: Connect to your Google Sheet
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1_QmYN7ac26U5mzW-fbgmxCKTeOEd9MebsfAO6GIS_Uk/edit"
sheet = gc.open_by_url(spreadsheet_url).sheet1

## Step 4: Fetch all sheet data
# Skip the header (first row)
questions  = sheet.col_values(1)[1:]
answers    = sheet.col_values(2)[1:]
timestamps = sheet.col_values(3)[1:]

num_rows = len(questions)
latest_index = None

# Step 5: Identify the latest new question (with no answer or timestamp)
for i in reversed(range(num_rows)):
    q = questions[i].strip()
    a = answers[i].strip() if i < len(answers) else ""
    t = timestamps[i].strip() if i < len(timestamps) else ""

    if q and not a and not t:
        latest_index = i
        break

## Step 6: Define LangChain RunnableSequence for clean chaining

def check_if_question_needs_answer(row_dict):
    if row_dict["question"] and not row_dict["answer"] and not row_dict["timestamp"]:
        return row_dict["question"]
    return None  # Skip

def generate_answer(question):
    return {
        "answer": qa_chain.run(question),
        "timestamp": datetime.now(pytz.timezone("America/New_York")).strftime("%Y-%m-%d %H:%M:%S")

    }

# Define LangChain Runnable pipeline
check = RunnableLambda(check_if_question_needs_answer)
rag_chain = RunnableLambda(generate_answer)
pipeline = RunnableSequence(check | rag_chain)

## Step 7: Run pipeline only if there is a new question
if latest_index is not None:
    latest_q = questions[latest_index]
    row_data = {
        "question": latest_q,
        "answer": answers[latest_index] if latest_index < len(answers) else "",
        "timestamp": timestamps[latest_index] if latest_index < len(timestamps) else ""
    }

    try:
        result = pipeline.invoke(row_data)
        sheet.update_cell(latest_index + 2, 2, result["answer"])       # Column B
        sheet.update_cell(latest_index + 2, 3, result["timestamp"])    # Column C

        print("✅ Latest Legal Question Answered:")
        print(f"\n🟨 Question: {latest_q}\n🟩 Answer: {result['answer']}")

    except Exception as e:
        sheet.update_cell(latest_index + 2, 4, f"Error: {str(e)}")     # Column D for errors
        print(f"❌ Error: {str(e)}")
else:
    print("✅ No new unanswered questions found.")


✅ Latest Legal Question Answered:

🟨 Question: who makes repairs inside the unit if theres a damage ?
🟩 Answer:  According to the context provided, it is the responsibility of the landlord to make repairs inside the unit if there is damage. The tenant must first notify the landlord of the situation and allow a reasonable amount of time for the landlord to make repairs or replacements. If the landlord fails to take action, the tenant may have the repairs made and deduct the cost from future rent. However, the tenant may still be taken to court for nonpayment of rent.


## ✅ MVP Summary: Legal RAG Chatbot with Google Sheets Integration  
This MVP answers user-submitted legal questions by applying Retrieval-Augmented Generation (RAG) on a legal PDF. The entire flow is orchestrated using LangChain and responses are written back to a Google Sheet.

### 🔍 What It Does:
- Loads and chunks a legal document (e.g., *Tenant Rights*) using LangChain utilities  
- Creates vector embeddings and stores them in a FAISS index  
- Detects newly added questions in a Google Sheet  
- Retrieves relevant PDF chunks and generates contextual answers using an LLM  
- Writes the answer and timestamp back to the sheet without modifying prior entries

### 🧰 Tech Stack:
- Google Colab + Google Sheets API (`gspread`)  
- LangChain + `langchain-openai`  
- OpenAI Embeddings + GPT-based LLM  
- FAISS for semantic document retrieval  
- Python `RunnableSequence` for modular flow control

### 🎯 Product Thinking:
- Enables legal teams or citizens to get contextual document-backed answers in a familiar spreadsheet format  
- Mimics customer service or self-serve Q&A tools without needing a full app  
- Designed for low-friction upgrades — e.g., swapping the sheet trigger for a web form or API
